<a href="https://colab.research.google.com/github/AvinivA/DSL-project/blob/main/DSL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install contractions
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os.path
import wget
import shutil

import numpy as np
import pandas as pd

# Expands English contractions in Text (I'd -> I would)
# https://pypi.org/project/contractions/
import contractions

import re  # library enabling the use of REGEX Operations to filter unwanted characters

import tensorflow as tf

import tensorflow.keras as keras
import tensorflow_hub as hub

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/DSL project/train_reviews.csv')

In [6]:
train_df.head()

,review_title,review_body,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID
0,Super product,"My husband has a hard time sleeping, the very ...",2020-04-13,5,0,0,380263,1,1,389,34510
1,Very sturdy,Rving,2020-08-29,5,0,0,845137,0,0,99,381688
2,Five Stars,has dramatically improved my toe nail appearances,2018-02-27,5,0,0,659759,0,0,107,417933
3,Great portable irrigator,I recently got braces and needed this for work...,2012-05-16,5,0,1,73493,0,0,66,166465
4,Four Stars,The Salad Shooter works great and saves time.,2015-01-05,4,0,0,714841,0,0,14,289006


In [7]:
train_df['fake_review'].value_counts()[1]

20065

In [8]:
def preprocess(text):  # Data Preprocessing
    text = str(text)
    text = text.lower()  # All text must be converted to lowercase to reduce duplicate detection and processing of the
    # same word
    text = contractions.fix(text)  # Expands all English contractions to reduce duplicate detection and processing of
    # the same expression (who's -> who is)

    # Filtering unwanted characters via Regular Expressions
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\d", " ", text)
    text = re.sub(r"\s+[a-z]\s+", " ", text)
    text = re.sub(r"^[a-z]\s+", " ", text)
    text = re.sub(r"\s+[a-z]$", " ", text)
    text = re.sub(r"\s+", " ", text)

    return text

In [9]:
train_df["review_body"] = train_df['review_body'].apply(lambda x: preprocess(x))

In [10]:
all_cols = train_df.columns
req_cols = ['review_body','fake_review']

train_shortened_df = train_df[req_cols]

In [11]:
train_shortened_df.head()

,review_body,fake_review
0,my husband has hard time sleeping the very fir...,1
1,rving,0
2,has dramatically improved my toe nail appearances,0
3,recently got braces and needed this for work ...,0
4,the salad shooter works great and saves time,0


In [12]:
# Training is done on 80%, validation on 10% and test on remining 10% data.
x_train, x_testval, y_train, y_testval = train_test_split(train_shortened_df[['review_body']], train_shortened_df[['fake_review']] , test_size = 0.20)
x_test, x_val, y_test, y_val = train_test_split(x_testval[['review_body']], y_testval[['fake_review']] , test_size = 0.50)

In [13]:
type(y_train)

pandas.core.frame.DataFrame

In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
print(f'{x_train.shape},{y_train.shape}')
(y_train['fake_review']).value_counts()

(314208, 1),(314208, 1)


0    298044
1     16164
Name: fake_review, dtype: int64

Need to vectorize before using the smote on numerical data

In [16]:
x_train

,review_body
295215,it is nice it works as it is supposed too do n...
336375,the front part had very noticeable stain my da...
186979,great swimming pool our community pool was clo...
253656,like it it will be better if charging port is...
287611,was gift for friends daughter and she just lov...
...,...
85327,after separating the top from the cable cover ...
322664,finally an organizer with cover
131418,sounds very good but espected more
311124,perfect product instead of wiping and vacuumin...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(x_train.values.ravel())
X_train=vectorizer.transform(x_train.values.ravel())
X_test=vectorizer.transform(x_test.values.ravel())
X_train=X_train.toarray()
X_test=X_test.toarray()

In [ ]:
x_train

In [36]:
smt = SMOTE()
x_train_sm, y_train_sm = smt.fit_resample(x_train,y_train)

ValueError: ignored

In [ ]:
print(x_train_sm.shape(),y_train_sm.shape()) 

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2" # pre-trained Tensoflow Hub Model

# trainable=False means we have frozen the weights of the pre-trained model, no training will happen on these weights for now
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=False) 

model_classifier = tf.keras.Sequential()
model_classifier.add(hub_layer)  # Adding pre-trained Tensorflow Hub Model to our own model

model_classifier.add(Dense(16, activation='relu'))

# 1D outputting final layer with Sigmoid Activation Function to predict Fake Reviews (LABEL) in a Binary Fashion
model_classifier.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model_classifier.compile(optimizer='adam',
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.metrics.BinaryAccuracy(threshold=0.5, name='acc')])

In [ ]:
model_classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 833
Non-trainable params: 48,190,600
_________________________________________________________________


In [ ]:
train_df.shape

(392760, 11)

In [ ]:
history_frozen_weights = model_classifier.fit(x_train,
                            y_train,
                            epochs=30,
                            batch_size=65460,
                            validation_data=(x_val, y_val),
                            verbose=1)

model_classifier.save("fakeReviewDetector_frozen_weights.hdf5")

Epoch 1/30
5/5 [==============================] - 7s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 2/30
5/5 [==============================] - 6s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 3/30
5/5 [==============================] - 6s 1s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 4/30
5/5 [==============================] - 6s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 5/30
5/5 [==============================] - 7s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 6/30
5/5 [==============================] - 6s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 7/30
5/5 [==============================] - 6s 2s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - val_acc: 0.9486
Epoch 8/30
5/5 [==============================] - 4s 1s/step - loss: 0.1986 - acc: 0.9491 - val_loss: 0.2002 - 

In [ ]:
results = model_classifier.evaluate(x_test[['review_body']],  y_test["fake_review"])

print(f'Evaluation on all data has accuracy of: {results[1]} ')

1228/1228 [==============================] - 4s 3ms/step - loss: 0.2031 - acc: 0.9477
Evaluation on all data has accuracy of: 0.9476525187492371 


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/DSL project/test_reviews.csv')

In [ ]:
test_df.head()

,review_title,review_body,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,product_ID,review_ID
0,Fit of phone 7+,I have the iPhone 7+ and it says it fits this....,2019-09-04,4,1,0,440934,0,281,427196
1,Sturdy and simply,This product comes assembled and works simply ...,2020-08-22,5,0,0,1044859,0,17,205467
2,Great product,I really like this product because 1st my kids...,2020-05-29,5,0,0,377805,1,342,202189
3,Item got good reviews but ...,I couldn't get the product to stay in my ear. ...,2019-09-07,1,0,2,747580,0,212,56726
4,Great Sheet Protectors - Love the Easy Loading,These sheet protectors were the perfect weight...,2012-10-05,5,0,0,1170571,0,10,409558


In [ ]:
test_df["review_body"] = test_df['review_body'].apply(lambda x: preprocess(x))

In [ ]:
yhat = model_classifier.predict(test_df["review_body"] )

1364/1364 [==============================] - 4s 3ms/step


In [ ]:
yhat = model_classifier.predict(x_val)

1228/1228 [==============================] - 6s 4ms/step


In [ ]:
yhat

array([[0.0572777 ],
       [0.06663977],
       [0.05878381],
       ...,
       [0.04273963],
       [0.05866301],
       [0.06973474]], dtype=float32)

In [ ]:
data['name'].value_counts()['sravan']

fake_review    39276
dtype: int64

In [ ]:
count = 0
for i in range(len(yhat)):
  if yhat[i] > 0.5:
    count += 1
count

0

In [ ]:
yhat

0